In [10]:
import pandas as pd
import numpy as np
import warnings
from functions import metrics
import csv
from sklearn.exceptions import ConvergenceWarning
from functions.formatting import get_subgroup_str
from itertools import product, combinations
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [11]:
omit_demographics = True 
use_gerryfair = False 

In [12]:
df = pd.read_csv('./data/preprocessed.csv')
df.drop(['umich_user_id', 'Unnamed: 0'], axis=1, inplace=True)

In [13]:
X = df.drop('completed', axis=1)
y = df['completed']

#### Calculate false positive rates for the given subgroup

This function takes in the subgroup to test for as a list of tuples. We use logistic regression and 10-fold cross validation to calculate the average FPR and AUC for the given subgroup. 

Returns a tuple containing FPR average, FPR standard deviation, AUC average, AUC standard deviation, and number of students in a given subgroup

#### Compute all subgroups

This function generates all possible subgroups given the list of protected columns

In [14]:
demo_groups = [['US', 'intl'], ['bachelor_obtained', 'no_bachelor_obtained', 'education_na'], ['white', 'black', 'asian', 'latinx', 'race_others', 'race_na'], ['male', 'female', 'gender_na', 'gender_other']]

In [15]:
def compute_combos():
    ret = []
    for i in range(1,len(demo_groups)+1):
        g = list(combinations(demo_groups, i))
        for demos in g:
            ret += list(product(*demos))

    return ret

In [16]:
combos = compute_combos()

In [17]:
combos.sort(key=len)
# combos
combos.insert(0, ())

In [18]:
combos

[(),
 ('US',),
 ('intl',),
 ('bachelor_obtained',),
 ('no_bachelor_obtained',),
 ('education_na',),
 ('white',),
 ('black',),
 ('asian',),
 ('latinx',),
 ('race_others',),
 ('race_na',),
 ('male',),
 ('female',),
 ('gender_na',),
 ('gender_other',),
 ('US', 'bachelor_obtained'),
 ('US', 'no_bachelor_obtained'),
 ('US', 'education_na'),
 ('intl', 'bachelor_obtained'),
 ('intl', 'no_bachelor_obtained'),
 ('intl', 'education_na'),
 ('US', 'white'),
 ('US', 'black'),
 ('US', 'asian'),
 ('US', 'latinx'),
 ('US', 'race_others'),
 ('US', 'race_na'),
 ('intl', 'white'),
 ('intl', 'black'),
 ('intl', 'asian'),
 ('intl', 'latinx'),
 ('intl', 'race_others'),
 ('intl', 'race_na'),
 ('US', 'male'),
 ('US', 'female'),
 ('US', 'gender_na'),
 ('US', 'gender_other'),
 ('intl', 'male'),
 ('intl', 'female'),
 ('intl', 'gender_na'),
 ('intl', 'gender_other'),
 ('bachelor_obtained', 'white'),
 ('bachelor_obtained', 'black'),
 ('bachelor_obtained', 'asian'),
 ('bachelor_obtained', 'latinx'),
 ('bachelor_obt

In [29]:
demographics = ['US', 'intl', 'bachelor_obtained', 'no_bachelor_obtained', 'education_na', 'white', 'black', 'asian', 'latinx', 'race_others', 'race_na', 'male', 'female', 'gender_na', 'gender_other']
protected=['US', 'intl', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female']

In [20]:
res = metrics.calc_metrics(X, y, combos, omit_demographics=False, demographics=demographics, protected=protected, is_gerryfair=False, iters=3)

c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\a006

In [21]:
# res

Generate a list of all subgroup data and write it to a csv

In [22]:
csv_file = f"./data/MTC508_subgroup_data_with_demographics.csv"
# csv_file = "test.csv"

# Write the data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    fieldnames = [
        'subgroup', 'n', 'auc_avg', 'auc_std', 'fpr_avg', 'fpr_std', 'rmse_avg', 'rmse_std']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()  # Write the header row
    for row in res:
        writer.writerow(row)

In [23]:
# data = {}

# data['overall'] = len(X)

# for c in combos[1:]:
#     masks = [X[name] == value for name, value in c]
#     final_mask = pd.concat(masks, axis=1).all(axis=1)
#     filter_X = X[final_mask]
#     data[get_subgroup_str(c)] = len(filter_X)




In [24]:
# df = pd.DataFrame(list(data.items()), columns=['Key', 'Count'])

# # Specify the CSV file path
# csv_file_path = 'data/MTC508_data_counts.csv'

# # Write the DataFrame to a CSV file
# df.to_csv(csv_file_path, index=False)

In [26]:
subgroup_data = pd.read_csv('./data/MTC508_subgroup_data_without_demographics.csv')
fpr_data = {}
# loop through each subgroup
# for each subgroup, get the corresponding data
for index, row in subgroup_data.iterrows():
    # print(f"Index: {index}, fpr: {row['fpr_avg']}")
    fpr_data[row['subgroup']] = row['fpr_avg']



In [30]:
def get_data_iters(start, stop, step, attrs):
    ret_val = []
    data = []
    for i in range(start+1,stop+1,step):
        data.append(metrics.calc_metrics(X, y, combos, omit_demographics=True, demographics=demographics, protected=protected, is_gerryfair=True, iters=i))
    
    for a in attrs:
        fprs = []
        for res in data:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break

        iterations = np.array(list(range(start, stop, step)))

        other_algorithm_fpr = np.linspace(fpr_data[a], fpr_data[a], iterations.size)

        ret_val.append((iterations, fprs, other_algorithm_fpr))
    
    return ret_val
        


In [31]:
val = get_data_iters(0, 3, 1, ['black', 'female'])

c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\a006

iteration: 1
most accurate classifier accuracy: 0.2121546961325967, most acc-class unfairness: 0.027314436704299786, most acc-class size 0.3348066298342542
iteration: 1
most accurate classifier accuracy: 0.19889502762430938, most acc-class unfairness: 0.022354441138971528, most acc-class size 0.3535911602209945
iteration: 1
most accurate classifier accuracy: 0.2196467991169978, most acc-class unfairness: 0.02874731167514615, most acc-class size 0.33885209713024284
iteration: 1
most accurate classifier accuracy: 0.2152317880794702, most acc-class unfairness: 0.02581083375785363, most acc-class size 0.33995584988962474
iteration: 1
most accurate classifier accuracy: 0.22185430463576158, most acc-class unfairness: 0.027056465667479962, most acc-class size 0.33885209713024284


c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\a006x2\Desktop\gerryfair\algofair\.venv\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


iteration: 1
most accurate classifier accuracy: 0.2121546961325967, most acc-class unfairness: 0.027314436704299786, most acc-class size 0.3348066298342542
iteration: 2
iteration: 1
most accurate classifier accuracy: 0.19889502762430938, most acc-class unfairness: 0.022354441138971528, most acc-class size 0.3535911602209945
iteration: 2


In [ ]:
def build_graph(x_axis, y_axis, other_algorithm_fpr, x_axis_label, y_axis_label, a):
        plt.plot(x_axis, y_axis, marker='o', linestyle='-', label='GerryFair')

        plt.plot(x_axis, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel(x_axis_label)
        plt.ylabel(y_axis_label)
        plt.title(f'Comparison of Algorithms - {y_axis_label} vs {x_axis_label} For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
attributes = ["Overall", "Black", "Female", "Black + Female"]

for i, attr in enumerate(attributes):
    build_graph(val[i][0], val[i][1], val[i][2], 'Iterations', 'FPR', attr)

NameError: name 'val' is not defined

In [ ]:
def build_graph_gamma(iters, gammas, attrs):
    results = []
    for g in gammas:
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=iters+1, gamma=g))
    
    data = results
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        gammas = np.array(gammas)

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], gammas.size) 

        plt.plot(gammas, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(gammas, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Gamma')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Gamma For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_gamma(5, [.002, .005, .01, .02, .05, .1], ['Overall', 'black', 'female', 'black, female'])

In [ ]:
len(data)

In [ ]:
def build_graph_attrs(iters, attrs):
    results = []
    for i in range(len(demographics)):
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=demographics[:i+1], is_gerryfair=True, iters=iters+1, gamma=g))
    
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        num_attrs = np.array(list(range(1,len(demographics) + 1)))

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], len(demographics)) 

        plt.plot(num_attrs, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(num_attrs, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Number of Attributes')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Number of Attributes For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_attrs(5, ['Overall', 'black', 'female', 'black, female'])